In [10]:
import pandas as pd

ebay_df = pd.read_csv("ebay_df.csv")
stock_df = pd.read_csv("stock_df.csv")
store_df = pd.read_csv("store_df.csv")
product_df = pd.read_csv("product_df.csv")
ams_df = pd.read_csv("store_data/AMS.csv")

In [13]:
df_merged = pd.merge(product_df, ams_df, on="custom_label", how="inner")

In [14]:
df_merged

,product_id,part_number,custom_label,supplier,last_updated,item_id,brand,current_quantity
0,BMC-UKD-BMC-ACCORDKIT,UKD-BMC-ACCORDKIT,UKD-BMC-ACCORDKIT,BMC,2024-05-04 06:08:45,326137738117,BMC Kits,10
1,BMC-UKD-BMC-ACCORDKIT1,UKD-BMC-ACCORDKIT1,UKD-BMC-ACCORDKIT1,BMC,2024-05-04 06:08:45,326137738122,BMC Kits,10
2,BMC-UKD-BMC-ACCORDKIT2,UKD-BMC-ACCORDKIT2,UKD-BMC-ACCORDKIT2,BMC,2024-05-04 06:08:45,335406938300,BMC Kits,10
3,BMC-UKD-BMC-ACCORDKIT3,UKD-BMC-ACCORDKIT3,UKD-BMC-ACCORDKIT3,BMC,2024-05-04 06:08:45,326137738116,BMC Kits,10
4,BMC-UKD-BMC-ACCORDKIT4,UKD-BMC-ACCORDKIT4,UKD-BMC-ACCORDKIT4,BMC,2024-05-04 06:08:45,326137738124,BMC Kits,10
...,...,...,...,...,...,...,...,...
44681,RTG-WWY55,WWY55,WWY55,RTG,2024-05-04 06:08:45,226159361991,Ikio,20
44682,RTG-WWY55,WWY55,WWY55,RTG,2024-05-04 06:08:45,226159361992,Ikio,20
44683,RTG-WWY55,WWY55,WWY55,RTG,2024-05-04 06:08:45,335406942655,Ikio,20
44684,RTG-WWY8X2,WWY8X2,WWY8X2,RTG,2024-05-04 06:08:45,324388393577,Base AMS,20


WWY55
UKD-BMC-ACCORDKIT

In [2]:
ebay_df


,Action,CustomLabel,SiteID,Currency,Quantity
0,Revise,UKD-APE-ABR101,UK,GBP,10
1,Revise,UKD-APE-ABR102,UK,GBP,10
2,Revise,UKD-COM-CS17,UK,GBP,3


In [3]:
stock_df

,product_id,quantity_delta,updated_date,Quantity
0,APE-ABR101,5.0,2024-08-06,10.0
1,APE-ABR102,5.0,2024-08-06,10.0
2,COM-CS17,1.0,2024-08-06,3.0


In [4]:
product_df[product_df["custom_label"] == "CS17"]

,product_id,part_number,custom_label,supplier,last_updated
216594,RTG-CS17,CS17,CS17,RTG,2024-05-04 06:08:45


In [5]:
df = pd.merge(store_df, product_df, on="custom_label", how="left")

In [6]:
df[df["custom_label"] == "CS17"]

,item_id,brand,custom_label,current_quantity,store,product_id,part_number,supplier,last_updated
3665,203606677155,Base AST,CS17,20,AST,RTG-CS17,CS17,RTG,2024-05-04 06:08:45
3666,154933880053,Unova,CS17,20,AST,RTG-CS17,CS17,RTG,2024-05-04 06:08:45
3667,154933880056,Unova,CS17,20,AST,RTG-CS17,CS17,RTG,2024-05-04 06:08:45
3668,154933880057,Unova,CS17,20,AST,RTG-CS17,CS17,RTG,2024-05-04 06:08:45
3669,154933880059,Unova,CS17,20,AST,RTG-CS17,CS17,RTG,2024-05-04 06:08:45
...,...,...,...,...,...,...,...,...,...
640325,335406890779,Ikio,CS17,20,AMS,RTG-CS17,CS17,RTG,2024-05-04 06:08:45
640326,335406890769,Ikio,CS17,20,AMS,RTG-CS17,CS17,RTG,2024-05-04 06:08:45
640327,335406890786,Ikio,CS17,20,AMS,RTG-CS17,CS17,RTG,2024-05-04 06:08:45
640328,335406890863,Ikio,CS17,20,AMS,RTG-CS17,CS17,RTG,2024-05-04 06:08:45


In [7]:
df = pd.merge(df, stock_df, on="product_id", how="inner")

In [8]:
df

,item_id,brand,custom_label,current_quantity,store,product_id,part_number,supplier,last_updated,quantity_delta,updated_date,Quantity
0,155321963392,Apec,UKD-APE-ABR101,10,AST,APE-ABR101,ABR101,APE,2024-05-04 06:08:45,5.0,2024-08-06,10.0
1,155322066970,Base AST,UKD-APE-ABR101,10,AST,APE-ABR101,ABR101,APE,2024-05-04 06:08:45,5.0,2024-08-06,10.0
2,204194607302,Unova,UKD-APE-ABR101,10,AST,APE-ABR101,ABR101,APE,2024-05-04 06:08:45,5.0,2024-08-06,10.0
3,204193670501,Apec,UKD-APE-ABR102,10,AST,APE-ABR102,ABR102,APE,2024-05-04 06:08:45,5.0,2024-08-06,10.0
4,204193760697,Base AST,UKD-APE-ABR102,10,AST,APE-ABR102,ABR102,APE,2024-05-04 06:08:45,5.0,2024-08-06,10.0
5,155323469175,Unova,UKD-APE-ABR102,10,AST,APE-ABR102,ABR102,APE,2024-05-04 06:08:45,5.0,2024-08-06,10.0
6,204691016501,Comline,UKD-COM-CS17,2,AST,COM-CS17,CS17,COM,2024-05-04 06:08:45,1.0,2024-08-06,3.0
7,156115447863,Base AST,UKD-COM-CS17,2,AST,COM-CS17,CS17,COM,2024-05-04 06:08:45,1.0,2024-08-06,3.0
8,156115547819,Unova,UKD-COM-CS17,2,AST,COM-CS17,CS17,COM,2024-05-04 06:08:45,1.0,2024-08-06,3.0
